In [1]:
import numpy as np

# Функии логики

На примере взаимодействия отдельных нейронов с пороговой функцией активации разберемся, как нейронные сети аппроксимируют простейшие булевы функцие, такие как **equal, not, and, or, xor, nor**

## Пороговая функция

$$
f(x_i) = \begin{cases} 0,\; x_i < 0; \\ 1, else.\end{cases}
$$

Нейрон:
$$
y = f(W\mathbf{x} + b)
$$

Реализуем пороговую функцию и нейрон (:

In [2]:
class HeavisideStepFunction:
    def __call__(self, x):
        """
        Heaviside step function.
        Takes a scalar and returns the activation.
        """
        if x < 0:
            return 0
        else:
            return 1

In [3]:
class SimpleNeuron:
    def __init__(self, n_in=2, activation=True):
        self.weights = np.zeros(n_in)
        self.bias = 0
        if activation:
            self.activation = HeavisideStepFunction()
        else:
            self.activation = None
        
    def set_weights(self, w, b):
        if len(self.weights) == 1:
            w = w * np.ones(1)
        self.weights = w
        self.bias = b
        
    def __call__(self, x):
        lin_comb = self.weights @ (x * np.ones(len(self.weights)))
        return self.activation(lin_comb + self.bias)
        

Теперь наша задача состоит в том, чтобы задать такие параметры нейрона, чтобы его выходы соответствовали таблице истинности данной функции логики. 

Ниже, в качестве примера, приведена функция тождества, которая не меняет аргументы булевой функции. 

### Equal 

In [4]:
neuron = SimpleNeuron(n_in=1)

In [5]:
neuron.set_weights(1, -0.5)

In [6]:
assert neuron(1.) == 1
assert neuron(0.) == 0

А дальше сами :) 

### Not

In [7]:
neuron = SimpleNeuron(n_in=1)

In [8]:
neuron.set_weights(-1, 0.5)

In [9]:
assert neuron(1.) == 0
assert neuron(0.) == 1

### Or

In [10]:
neuron = SimpleNeuron(n_in=2)

In [11]:
neuron.set_weights(np.array([1, 1]), -0.5)

In [12]:
assert neuron(np.array([0, 0])) == 0
assert neuron(np.array([0, 1])) == 1
assert neuron(np.array([1, 0])) == 1
assert neuron(np.array([1, 1])) == 1

### And

In [13]:
neuron = SimpleNeuron(n_in=2)

In [14]:
neuron.set_weights(np.array([1, 1]), -1.5)

In [15]:
assert neuron(np.array([0, 0])) == 0
assert neuron(np.array([0, 1])) == 0
assert neuron(np.array([1, 0])) == 0
assert neuron(np.array([1, 1])) == 1

### Nor

In [16]:
neuron = SimpleNeuron(n_in=2)

In [21]:
neuron.set_weights(np.array([-1, -1]), 0)

In [22]:
assert neuron(np.array([0, 0])) == 1
assert neuron(np.array([0, 1])) == 0
assert neuron(np.array([1, 0])) == 0
assert neuron(np.array([1, 1])) == 0

### NAND

In [31]:
neuron = SimpleNeuron(n_in=2)

In [32]:
neuron.set_weights(np.array([-1, -1]), 1)

In [33]:
assert neuron(np.array([0, 0])) == 1
assert neuron(np.array([0, 1])) == 1
assert neuron(np.array([1, 0])) == 1
assert neuron(np.array([1, 1])) == 0

### Implication 

In [23]:
neuron = SimpleNeuron(n_in=2)

In [24]:
neuron.set_weights(np.array([-1, 2]), 0)

In [25]:
assert neuron(np.array([0, 0])) == 1
assert neuron(np.array([0, 1])) == 1
assert neuron(np.array([1, 0])) == 0
assert neuron(np.array([1, 1])) == 1

### Xor

Можно ли реализовать xor одним нейроном? Двумя? Объясните почему?

__Proof.__
We need to solve the next system of inequalities for 1 neuron:
$$
\begin{cases}
c < 0, \\
b + c \geqslant 0, \\
a + c \geqslant 0, \\
a + b + c < 0.
\end{cases}
$$
Solving this system we obtain that $0 \leqslant a + c + b + c < c < 0.$ This is a contradiction! 

In [34]:
class XorNet:
    def __init__(self):
        self.neuron_1 = SimpleNeuron(n_in=2)
        self.neuron_2 = SimpleNeuron(n_in=2)
        self.neuron_3 = SimpleNeuron(n_in=2)

    def __call__(self, x):
        return self.neuron_3(np.array([self.neuron_1(x), self.neuron_2(x)]))

In [35]:
xor = XorNet()

In [36]:
xor.neuron_1.set_weights(np.array([1, 1]), -0.5)  # OR
xor.neuron_2.set_weights(np.array([-1, -1]), 1)   # NAND
xor.neuron_3.set_weights(np.array([1, 1]), -1.5) # AND

In [39]:
assert xor(np.array([0, 0])) == 0
assert xor(np.array([0, 1])) == 1
assert xor(np.array([1, 0])) == 1
assert xor(np.array([1, 1])) == 0

Предложите активацию, для которой **xor** реализуется одним нейроном?

In [ ]:
# your code here

## Универсальная теорема об аппроксимации. 

Теперь рассмотрим более общую задачу, аппроксимации не булевой функции. 


Пусть $\varphi$  любая непрерывная сигмоидная функция, например, $\varphi (\xi )=1/(1+e^{-\xi })$.

 Тогда, если дана любая непрерывная функция действительных переменных $f$ на $[0,1]^{n}$ (или любое другое компактное подмножество $\mathbb {R} ^{n}$) и $\varepsilon >0$, то существуют векторы 
 $$\mathbf {w_{1}} ,\mathbf {w_{2}} ,\dots ,\mathbf {w_{N}} ,\mathbf {\alpha }, \mathbf {\theta } $$
 
  и параметризованная функция 
  $$G(\mathbf {\cdot } ,\mathbf {w} ,\mathbf {\alpha } ,\mathbf {\theta } ):[0,1]^{n}\to R$$
    такая, что для всех $\mathbf {x} \in [0,1]^{n}$ выполняется

$$ {\big |}G(\mathbf {x} ,\mathbf {w} ,\mathbf {\alpha } ,\mathbf {\theta } )-f(\mathbf {x} ){\big |}<\varepsilon ,$$
где

$$ G(\mathbf {x} ,\mathbf {w} ,\mathbf {\alpha } ,\mathbf {\theta } )=\sum _{i=1}^{N}\alpha _{i}\varphi (\mathbf {w} _{i}^{T}\mathbf {x} +\theta _{i}), $$

